In [0]:
%sql
MERGE INTO workspace.gold_staging.stg_sst AS target
USING (
  SELECT * FROM (
    SELECT
      silver.time,
      sst_item.fk,
      sst_item.sst_res.number,
      sst_item.sst_res.in_num,
      sst_item.sst_res.in_name,
      sst_item.sst_res.sst_name,
      sst_item.sst_res.peak,
      sst_item.sst_res.eval_result,
      sst_item.sst_res.result,
      ROW_NUMBER() OVER (
        PARTITION BY silver.time, sst_item.fk
        ORDER BY silver.time
      ) AS rn
    FROM workspace.silver_schema.silver_delta_table AS silver
    LATERAL VIEW EXPLODE(silver.sst) AS sst_item
  ) WHERE rn = 1
) AS source
ON target.fk = source.fk
AND target.time = source.time
WHEN MATCHED THEN
  UPDATE SET
    target.number = source.number,
    target.in_num = source.in_num,
    target.in_name = source.in_name,
    target.sst_name = source.sst_name,
    target.peak = source.peak,
    target.eval_result = source.eval_result,
    target.result = source.result
WHEN NOT MATCHED THEN
  INSERT (
    time,
    fk,
    number,
    in_num,
    in_name,
    sst_name,
    peak,
    eval_result,
    result
  )
  VALUES (
    source.time,
    source.fk,
    source.number,
    source.in_num,
    source.in_name,
    source.sst_name,
    source.peak,
    source.eval_result,
    source.result
  );
